In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool #pip install pathos
import re
import pymysql
def scoll(driver):
        SCROLL_PAUSE_TIME = 1
        last_height = driver.execute_script("return document.body.scrollHeight")        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)                                                
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:                                                
                break
            last_height = new_height
class Parser():
    def __init__(self):
        self.pool = Pool(processes=3)
        
    def open_browser(self, site):
        driver = webdriver.Chrome('./chromedriver.exe')
        driver.get(site)
        scoll(driver)
        return driver.page_source
Free_Cancell=False
for i in range(3):
    i+=1
    url="https://www.myrealtrip.com/themes/1088/offers?order=recommended&page={}&per=32".format(i)
    headers={"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    driver = webdriver.Chrome()
    driver.get(url);
    scoll(driver)
    first_elems = driver.find_elements_by_class_name("OfferHorizontalCard-module__container--XWrxS.CardShare-module__container--lu1iP")
    print(len(first_elems))
    for elem in first_elems:
        parser = Parser()
        try:
            elem_temp = elem.find_element_by_class_name("CardFreeRefundSticker-module__container--W1Wtx")
            Free_Cancell = True 
        except:
            Free_Cancell = False 
        soup = BeautifulSoup(parser.open_browser(elem.get_attribute('href')), "lxml")
        name_temp=soup.find('h1', {'class':"OfferHeaderTitle-module__container--ZcKff"})
        jogun = re.compile(r'[^ -=+,#/?:^.*~ㆍ!A-Za-z0-9가-힣+]')
        name=jogun.sub('', name_temp.get_text())
        star_jum=soup.find('span', {'class':"OfferHeaderReview-module__score--PFNsE"})
        review_num=soup.find('span', {'class':"OfferHeaderReview-module__count--VmY1D"})
        price=soup.find('strong', {'class':"OfferPrice-module__price--rSHhS"})
        Wish_list=soup.find('p', {'class':"OfferSideBarMainWishlist-module__desc--W5of9"})

        country_city=soup.find_all('span', {'class':"OfferHeaderLocation-module__name--PhOlA"})
        country=country_city[0].get_text()
        city=country_city[1].get_text()
        db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
        ecommerce = db.cursor()
        sql = """INSERT INTO trip VALUES("{0}", "{1}", '''{2}''', {3}, {4}, {5}, {6},{7});""".format(country, city,name,re.sub(r'[^0-9]', '', price.get_text()), star_jum.get_text(),re.sub(r'[^0-9]', '', review_num.get_text()),re.sub(r'[^0-9]', '', Wish_list.get_text()),Free_Cancell)
        print(sql)
        ecommerce.execute(sql)
        db.commit()
        db.close()
    driver.close()
    driver.quit()   
    

In [ ]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
ecommerce = db.cursor()
sql = """
    create table trip (
        Country  varchar(30) not null,
        City  varchar(30),
        Package varchar(400),
        Price int,
        Rate DECIMAL(2,1),
        Review int,
        Wish int,
        Free_Cancell BOOLEAN 
    );
"""
ecommerce.execute(sql)
db.commit()
db.close()